In [1]:
import autograd.numpy as np

import wecopttool as wot

In [2]:
file = "./bem.nc"
data = wot.read_netcdf(file)
f1 = 0.05
nfreq = 50

In [3]:
amplitude = 0.0625  
idx = 6
freq = idx*f1
waves = wot.waves.regular_wave(f1, nfreq, freq, amplitude)

In [4]:
ndof_pto = 1
pto_names = ["PTO_Heave"]
kinematics = np.eye(ndof_pto)

controller = wot.pto.controller_unstructured
ncomponents_pto = 2*nfreq + 1
pto = wot.pto.PTO(ndof_pto, ncomponents_pto, kinematics, controller, pto_names)

In [5]:
data = wot.add_zerofreq_to_xr(data)
wec = wot.WEC.from_bem(data, f_add={'PTO': pto.force_on_wec})

Linear damping for DOF "HEAVE" has negative or close to zero terms. Shifting up via linear friction of 36.667232716402985 N/(m/s).


In [6]:
res = wec.solve(waves, pto.mechanical_average_power, pto.nstate)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -135.07292811011573
            Iterations: 35
            Function evaluations: 35
            Gradient evaluations: 35


In [7]:
w = data.coords['omega'][idx]
Z = ((1j*w)*(data['mass'] + data['added_mass'][0,0,idx])
     + data['radiation_damping'][0,0,idx]
     + (1/(1j*w)) * data['hydrostatic_stiffness']
     + 36.667232716402985
     )

exc_coeff = data['Froude_Krylov_force'] + data['diffraction_force']
F = wot.wave_excitation(exc_coeff, waves)[idx]

power_theoretical = (np.abs(F)**2 / (8 * Z.real)).values[0,0]
power = -res.fun

power, power_theoretical

(135.07292811011573, 135.0755469893718)